<a href="https://colab.research.google.com/github/sdgroeve/Machine_Learning_course_UGent_D012554_2025/blob/main/notebooks/overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Tree Overfitting Demonstration

This notebook demonstrates how decision trees can achieve 100% training accuracy but perform poorly on test data due to overfitting. We'll visualize this phenomenon using a synthetic non-linear dataset (half-moons) and explore how regularization techniques like limiting tree depth can help mitigate overfitting.

## What is Overfitting?

Overfitting occurs when a model learns the training data too well, capturing noise and outliers rather than just the underlying pattern. This results in:
- Excellent performance on training data
- Poor performance on new, unseen data (test data)

Decision trees are particularly prone to overfitting because they can create very complex decision boundaries that perfectly separate the training data.

First, let's import the necessary libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap

# Set random seed for reproducibility
np.random.seed(42)

## Data Generation

We'll generate a synthetic non-linear dataset using scikit-learn's `make_moons` function. This creates two interleaving half-moons, which is a classic example of data that requires non-linear decision boundaries.

To make overfitting more pronounced, we'll also add some noise and outliers by flipping the labels of a few random points.

In [ ]:
# Generate a non-linear dataset (half-moons)
X, y = make_moons(n_samples=500, noise=0.3, random_state=1)

# Add some noise/outliers to make overfitting more pronounced
# Add 20 random outliers
n_outliers = 20
outlier_indices = np.random.choice(len(X), n_outliers, replace=False)
# Flip the labels for these outliers
y[outlier_indices] = 1 - y[outlier_indices]

# Let's visualize our dataset
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=ListedColormap(['#FF0000', '#00FF00']), edgecolor='k', s=20)
plt.title('Half-moons Dataset with Outliers')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

## Data Splitting

We'll split our data into training (70%) and testing (30%) sets. The training set will be used to train our models, while the testing set will be used to evaluate their performance on unseen data.

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")

## Training an Unrestricted Decision Tree

First, let's train a decision tree with default parameters. By default, scikit-learn's `DecisionTreeClassifier` will grow a tree until all leaves are pure (contain samples of only one class) or until all leaves contain less than `min_samples_split` samples.

This allows the tree to potentially grow very deep and create a complex decision boundary that perfectly fits the training data, including noise and outliers.

In [ ]:
# Create a decision tree classifier with default parameters
# This allows tree to grow fully and potentially overfit
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)

# Training and testing accuracy
train_pred = tree_clf.predict(X_train)
test_pred = tree_clf.predict(X_test)

train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)

print(f"Training accuracy: {train_accuracy:.4f}")
print(f"Testing accuracy: {test_accuracy:.4f}")

## Training a Regularized Decision Tree

Now, let's train a regularized decision tree by limiting its maximum depth to 3. This prevents the tree from creating an overly complex decision boundary and forces it to focus on the most important patterns in the data.

In [ ]:
# Create a regularized tree with limited depth for comparison
reg_tree_clf = DecisionTreeClassifier(max_depth=3, random_state=42)
reg_tree_clf.fit(X_train, y_train)

reg_train_pred = reg_tree_clf.predict(X_train)
reg_test_pred = reg_tree_clf.predict(X_test)

reg_train_accuracy = accuracy_score(y_train, reg_train_pred)
reg_test_accuracy = accuracy_score(y_test, reg_test_pred)

print(f"Regularized tree:")
print(f"Training accuracy: {reg_train_accuracy:.4f}")
print(f"Testing accuracy: {reg_test_accuracy:.4f}")

## Visualizing Decision Boundaries

Let's define a function to visualize the decision boundaries of our models. This will help us see how the unrestricted tree creates a complex boundary that perfectly fits the training data, while the regularized tree creates a simpler boundary that generalizes better to unseen data.

In [ ]:
# Function to plot decision boundaries
def plot_decision_boundary(clf, X, y, title, ax=None):
    if ax is None:
        _, ax = plt.subplots(figsize=(8, 6))

    # Define bounds of the plot
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

    # Create a mesh grid
    h = 0.02  # Step size
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # Predict on the mesh grid
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Create custom colormap
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA'])
    cmap_bold = ListedColormap(['#FF0000', '#00FF00'])

    # Plot decision boundary and points
    ax.contourf(xx, yy, Z, cmap=cmap_light, alpha=0.8)
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold, edgecolor='k', s=20)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_title(title)

    return ax

Now, let's compare the decision boundaries of our unrestricted and regularized trees:

In [ ]:
# Create figure with subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot decision boundaries
plot_decision_boundary(tree_clf, X_train, y_train,
                       f"Overfitted Tree\nTrain: {train_accuracy:.4f}, Test: {test_accuracy:.4f}", ax1)
plot_decision_boundary(reg_tree_clf, X_train, y_train,
                       f"Regularized Tree (max_depth=3)\nTrain: {reg_train_accuracy:.4f}, Test: {reg_test_accuracy:.4f}", ax2)

ax1.legend()
ax2.legend()

fig.tight_layout()
plt.show()

## The Effect of Tree Depth on Overfitting

Let's explore how the maximum depth of the tree affects its performance on training and testing data. We'll train multiple trees with different maximum depths and plot their accuracies.

In [ ]:
# Parameter values to try
max_depths = range(1, 21)
train_scores = []
test_scores = []

for depth in max_depths:
    # Train tree with specific depth
    dt = DecisionTreeClassifier(max_depth=depth, random_state=42)
    dt.fit(X_train, y_train)

    # Record accuracies
    train_scores.append(accuracy_score(y_train, dt.predict(X_train)))
    test_scores.append(accuracy_score(y_test, dt.predict(X_test)))

# Plot accuracies vs tree depth
plt.figure(figsize=(10, 6))
plt.plot(max_depths, train_scores, 'o-', color='r', label='Training accuracy')
plt.plot(max_depths, test_scores, 'o-', color='g', label='Testing accuracy')

plt.axhline(y=1.0, color='r', linestyle=':', alpha=0.5, label='Perfect accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Max Tree Depth')
plt.title('Decision Tree Performance vs Tree Depth')
plt.legend()
plt.grid(True, alpha=0.3)

# Mark the point where overfitting starts to become severe
best_test_idx = np.argmax(test_scores)
plt.scatter([max_depths[best_test_idx]], [test_scores[best_test_idx]],
            s=200, facecolors='none', edgecolors='blue', linewidth=2,
            label=f'Best test accuracy at depth={max_depths[best_test_idx]}')

plt.legend()
plt.tight_layout()
plt.show()